Passive-Aggressive algorithms are generally used for large-scale learning. It is one of the few ‘online-learning algorithms‘. In online machine learning algorithms, the input data comes in sequential order and the machine learning model is updated step-by-step, as opposed to batch learning, where the entire training dataset is used at once. This is very useful in situations where there is a huge amount of data and it is computationally infeasible to train the entire dataset because of the sheer size of the data. We can simply say that an online-learning algorithm will get a training example, update the classifier, and then throw away the example.

## Let's start the work

In [ ]:
import pandas as pd

In [4]:
#dataframe = pd.read_csv('MyNews.csv')
dataframe = pd.read_csv(r'C:\Users\elGhizi\Desktop\fakeNews\FakeNews.txt', delimiter = ";")
dataframe.head()

,text,label
0,أشارت صحيفة واشنطن بوست إلى أن إدارة الرئيس ال...,REAL
1,يعتقد عدد كبير من الأمريكيين أن فيروس كورونا ت...,REAL
2,أعلنت وزارة الصحة الإيطالية تسجيل 77 وفاة جديد...,REAL
3,اعتبر رئيس الوزراء البريطاني بوريس جونسون أنه ...,REAL
4,أعلن المتحدث باسم الحكومة الفرنسية غابرييل أتا...,REAL


In [5]:
x = dataframe['text']
y = dataframe['label']

In [6]:
x

0       أشارت صحيفة واشنطن بوست إلى أن إدارة الرئيس ال...
1       يعتقد عدد كبير من الأمريكيين أن فيروس كورونا ت...
2       أعلنت وزارة الصحة الإيطالية تسجيل 77 وفاة جديد...
3       اعتبر رئيس الوزراء البريطاني بوريس جونسون أنه ...
4       أعلن المتحدث باسم الحكومة الفرنسية غابرييل أتا...
                              ...                        
1251    فريق تقصي الحقائق في بي بي سي يفند عدداً من ال...
1252    مع استمرار تفشي فيروس كورونا في مختلف انحاء ال...
1253    موقع تويتر يعلن عن إجراءات جديدة تهدف إلى الحد...
1254    كثير من المثقفين، بدءا من الطلاب ووصولا إلى ال...
1255    تنتشر عبر مواقع التواصل الاجتماعي أخبار كاذبة ...
Name: text, Length: 1256, dtype: object

In [7]:
y

0       REAL
1       REAL
2       REAL
3       REAL
4       REAL
        ... 
1251    FAKE
1252    FAKE
1253    FAKE
1254    FAKE
1255    FAKE
Name: label, Length: 1256, dtype: object

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [9]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
print("========x train============")
print(len(x_train))
print("=========x test===========")
print(len(x_test))
print("========y train============")
print(len(y_train))
print("========y test============")
print(len(y_test))

========x train============
1004
=========x test===========
252
========y train============
1004
========y test============
252


In [10]:
y_train

159     REAL
435     REAL
554     REAL
984     REAL
970     REAL
        ... 
763     REAL
835     REAL
1216    FAKE
559     REAL
684     REAL
Name: label, Length: 1004, dtype: object

In [11]:
tfvect = TfidfVectorizer(stop_words='arabic',max_df=0.7)
tfvect = TfidfVectorizer(max_df=0.8)
tfid_x_train = tfvect.fit_transform(x_train.values.astype('U'))
tfid_x_test = tfvect.transform(x_test.values.astype('U'))

* max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
* max_df = 25 means "ignore terms that appear in more than 25 documents".

In [12]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [33]:
y_pred = classifier.predict(tfid_x_test)
print(type(tfid_x_test))
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

<class 'scipy.sparse.csr.csr_matrix'>
Accuracy: 95.24%


In [28]:
cf = confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])
print(cf)

[[ 27   5]
 [  0 190]]


In [31]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    print(type(vectorized_input_data))
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [32]:
fake_news_det('لقاح فايزر يغيّر الحمض النووي البشري و يسبب مشاكل عصبية مثل الزهايمر')

<class 'scipy.sparse.csr.csr_matrix'>
['Fake']


In [17]:
fake_news_det("""Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)


['REAL']


In [18]:
import pickle
pickle.dump(classifier,open('model.pkl', 'wb'))

In [19]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [20]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [21]:
fake_news_det1('من اعراض كوفيد الحمى')

['REAL']


In [22]:
fake_news_det1("""U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.""")

['REAL']


In [23]:
fake_news_det('''U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.''')


['REAL']
